In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import shutil

['train_val_list.txt', 'images_007', 'BBox_List_2017.csv', 'images_011', 'LOG_CHESTXRAY.pdf', 'FAQ_CHESTXRAY.pdf', 'images_008', 'images_006', 'ARXIV_V5_CHESTXRAY.pdf', 'images_012', 'images_004', 'test_list.txt', 'Data_Entry_2017.csv', 'images_009', 'images_002', 'images_003', 'images_001', 'README_CHESTXRAY.pdf', 'images_005', 'images_010']


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [3]:
df = pd.read_csv('../input/Data_Entry_2017.csv')

    ## Uncomment

In [4]:
lst_files_001 = Path('../input/images_001/images').ls()
lst_files_002 = Path('../input/images_001/images').ls()
lst_files_003 = Path('../input/images_003/images').ls()
lst_files_004 = Path('../input/images_003/images').ls()
lst_files_005 = Path('../input/images_005/images').ls()
lst_files_006 = Path('../input/images_006/images').ls()
lst_files_007 = Path('../input/images_007/images').ls()
lst_files_008 = Path('../input/images_008/images').ls()
lst_files_009 = Path('../input/images_009/images').ls()
lst_files_010 = Path('../input/images_010/images').ls()
lst_files_011 = Path('../input/images_011/images').ls()
lst_files_012 = Path('../input/images_012/images').ls()


lst_files = lst_files_001 + lst_files_002 + lst_files_003 + lst_files_004 + lst_files_005 + lst_files_006 + lst_files_007+ lst_files_008 + lst_files_009 + lst_files_010 + lst_files_011 + lst_files_012

In [5]:
# Filtering the dataset for later use.
lst_names = list(map(os.path.basename, lst_files))
df_filtered = df[df['Image Index'].isin(lst_names)] # Filtered with the files which we are interested in.

In [6]:
def get_label(fname):
    file_name = (Path(os.path.basename(fname)))
    df_find = df.loc[df['Image Index'] == str(file_name)]
    labelcol = 'Finding Labels'
    retval = df_find[labelcol].values
    retval = ' '.join(map(str, retval))
    #print((retval))
    return((retval))

## Uncomment the following blocks for full training

In [7]:
il = ImageList.from_folder(path =  Path('../input/images_001/images'))
il = il.add(ImageList.from_folder(path =  Path('../input/images_002/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_003/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_004/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_005/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_006/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_007/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_008/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_009/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_010/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_011/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_012/images')))

In [8]:
sd = il.split_by_rand_pct(0.1) # split data
ll = sd.label_from_func(get_label, label_delim = "|")
data = ll.transform(size=128).databunch(bs = 16).normalize(imagenet_stats)

In [9]:
data

ImageDataBunch;

Train: LabelList (100908 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: MultiCategoryList
Atelectasis;Infiltration,No Finding,No Finding,No Finding,No Finding
Path: ../input/images_001/images;

Valid: LabelList (11212 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: MultiCategoryList
No Finding,Infiltration,No Finding,Infiltration;Nodule;Pneumothorax,No Finding
Path: ../input/images_001/images;

Test: None

In [10]:
#valid_filenames = data.valid_ds.items
#len(valid_filenames)

In [11]:
print(data.classes)

['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [12]:
#data.show_batch(rows=3, figsize=(15,15))

In [13]:
acc_05 = partial(accuracy_thresh, thresh=0.05)
f_05 = partial(fbeta, thresh=0.05)
acc_10 = partial(accuracy_thresh, thresh=0.1)
f_10 = partial(fbeta, thresh=0.1)
acc_15 = partial(accuracy_thresh, thresh=0.15)
f_15 = partial(fbeta, thresh=0.15)
acc_20 = partial(accuracy_thresh, thresh=0.2)
f_20 = partial(fbeta, thresh=0.2)

In [14]:
#p_thresh = .5

#def accuracy(input:Tensor, targs:Tensor)->Rank0Tensor:
    #"Computes accuracy with `targs` when `input` is bs * n_classes."
    #n = targs.shape[0]
   # input = input.argmax(dim=-1).view(n,-1)
    #targs = targs.view(-1).long()
    #return (input==targs).float().mean()

In [15]:
#arch = 
#acc_02 = partial(accuracy_thresh, thresh=p_thresh)
#f_score = partial(fbeta, thresh=p_thresh)
learn = cnn_learner(data, models.resnet50, metrics=[acc_05, f_05, acc_10, f_10, acc_15, f_15, acc_20, f_20],  model_dir="/tmp/model/")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 48.7MB/s]


In [16]:
#learn.lr_find()
#learn.recorder.plot()

In [17]:
import time
start_time = time.time()
learn.fit_one_cycle(5, 2 * 1e-2)
end_time = time.time()
print("Training time:", end_time - start_time, "seconds")

epoch,train_loss,valid_loss,accuracy_thresh,fbeta,accuracy_thresh,fbeta,accuracy_thresh,fbeta,accuracy_thresh,fbeta,time


In [18]:
#(learn.summary())

In [19]:
preds,y,losses = learn.get_preds(with_loss=True)
softmax = nn.Softmax(dim=-1)
preds = softmax(preds)
interp = ClassificationInterpretation(learn, preds, y, losses)

In [20]:
# looking for class with index = 0
#index = 0
#threshold = .2
#data.classes[index]
#vals = preds[:,0]; vals.shape
#output = vals > threshold
#output.size()
#output


In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, roc_curve, f1_score, accuracy_score
def get_acts_preds(preds, y, class_index, thresh = 0.2):
    vals = preds[:,class_index]; 
    nppreds = vals.numpy()
    y_pred = (nppreds > thresh).astype(int)
    y_true = (y[:,class_index].numpy() ==1).astype(int) 
    #print("accuracy:", )
    return(y_pred,y_true, accuracy_score(y_true, y_pred, normalize=True))

In [22]:
import pandas as pd
def do_roc_calc(preds, y, data_classes , df, class_index = 0):
    #df = pd.DataFrame(columns=['label_index', 'label', 'roc_auc_score', 'accuracy', 'accuracy_threshold'])
    df = pd.DataFrame()
    y_true = y[:,class_index].numpy()
    y_pred = preds[:,class_index].numpy()
    class_name = data_classes[class_index]
    print("***** ======== Error Analysis for:", class_name, class_index, " ======= *****")
    fp_rate, tp_rate, thresholds = roc_curve(y_true, y_pred)
    print("AUC:", auc(fp_rate, tp_rate, thresholds))
    score_roc_auc = roc_auc_score(y_true, y_pred)
    print("ROC area under curve: ", score_roc_auc)
    print("Printing Confusion Matrix: ")
   # vals = preds[:,class_index]; 
   # nppreds = vals.numpy()
   # predicted = (nppreds > thresh).astype(int)
   # actual = (y[:,class_index].numpy() ==1).astype(int) 
    thresh = 0.05
    predicted, actual, acc_05 = get_acts_preds(preds, y, class_index, thresh = thresh)
    print("Threshold:", thresh, "accuracy:", acc_05)
    print(confusion_matrix(actual, predicted))
    df = df.append({'label_index': class_index, 'label': class_name, 'roc-auc_score': score_roc_auc, 'acc_level': '.05', 'accuracy': acc_05 }, ignore_index=True)
    
    
    thresh = 0.1
    predicted, actual, acc_10 = get_acts_preds(preds, y, class_index, thresh = thresh)
    print("Threshold:", thresh, "accuracy:", acc_10)
    print(confusion_matrix(actual, predicted))
    df = df.append({'label_index': class_index, 'label': class_name, 'roc-auc_score': score_roc_auc, 'acc_level': '.10', 'accuracy': acc_10 }, ignore_index=True)
    
    thresh = 0.15
    predicted, actual, acc_15 = get_acts_preds(preds, y, class_index, thresh = thresh)
    print("Threshold:", thresh, "accuracy:", acc_15)
    print(confusion_matrix(actual, predicted))
    df = df.append({'label_index': class_index, 'label': class_name, 'roc-auc_score': score_roc_auc, 'acc_level': '.15', 'accuracy': acc_15 }, ignore_index=True)

    thresh = 0.2
    predicted, actual, acc_20 = get_acts_preds(preds, y, class_index, thresh = thresh)
    print("Threshold:", thresh, "accuracy:", acc_20)
    print(confusion_matrix(actual, predicted))
    df = df.append({'label_index': class_index, 'label': class_name, 'roc-auc_score': score_roc_auc, 'acc_level': '.20', 'accuracy': acc_20 }, ignore_index=True)

    
    #columns=['label_index', 'label', 'roc_auc_score', 'accuracy', 'accuracy_threshold']
    print("*****  End Analysis ************")
    return(df)

In [23]:
import warnings
warnings.filterwarnings("ignore")
df_result = pd.DataFrame()
for ci in list(range(15)):
    df =do_roc_calc(preds, y, data.classes , df_result, class_index = ci)
    df_result = df_result.append(df)
    #df_result.append(df)
print(df_result)

***** ======== Error Analysis for: Atelectasis 0  ======= *****
AUC: 0.7560147859806828
ROC area under curve:  0.7560147859806828
Printing Confusion Matrix: 
Threshold: 0.05 accuracy: 0.10149839457723868
[[    7 10074]
 [    0  1131]]
Threshold: 0.1 accuracy: 0.8988583660363896
[[10078     3]
 [ 1131     0]]
Threshold: 0.15 accuracy: 0.8991259364966108
[[10081     0]
 [ 1131     0]]
Threshold: 0.2 accuracy: 0.8991259364966108
[[10081     0]
 [ 1131     0]]
*****  End Analysis ************
***** ======== Error Analysis for: Cardiomegaly 1  ======= *****
AUC: 0.848784969087766
ROC area under curve:  0.848784969087766
Printing Confusion Matrix: 
Threshold: 0.05 accuracy: 0.02452729218694256
[[   10 10937]
 [    0   265]]
Threshold: 0.1 accuracy: 0.9763646093471281
[[10947     0]
 [  265     0]]
Threshold: 0.15 accuracy: 0.9763646093471281
[[10947     0]
 [  265     0]]
Threshold: 0.2 accuracy: 0.9763646093471281
[[10947     0]
 [  265     0]]
*****  End Analysis ************
***** =======

In [24]:
df_result.to_csv('error_info_2.csv', index=False)

In [25]:
#data.classes

In [26]:
#index = 0
#thresh = .2
#nppreds= preds.numpy()
#vals = preds[:,index]; 
#nppreds = vals.numpy()
#predicted = (nppreds > thresh).astype(int)
#c.shape
#actual = (y[:,index].numpy() ==1).astype(int) 
#res = (predicted == actual).astype(int)
#predicted

In [27]:
import random
def random_test(ds, lrn ):
    # find a random number between 0 to len(df)
    image_num = math.floor(random.uniform(0, len(ds)))
    img = ds[image_num][0]
    actual = ds[image_num][1]
    predict_2 = lrn.predict(img, thresh = 0.2)
    predict_3 = lrn.predict(img, thresh = 0.3)
    predict_4 = lrn.predict(img, thresh = 0.4)
    predict_5 = lrn.predict(img, thresh = 0.5)
    print("image num:", image_num)
    print("actual:", actual)
    print("predict_2:", predict_2)
    print("----------------------------------------")
    print("predict_3:", predict_3)
    print("----------------------------------------")
    print("predict_4:", predict_4)
    print("----------------------------------------")
    print("predict_5:", predict_5)
    print("----------------------------------------")
    

### Test on Train data

In [28]:
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)
random_test(data.train_ds, learn)

image num: 15170
actual: No Finding
predict_2: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([7.0797e-03, 1.7802e-03, 2.1385e-03, 3.2882e-04, 1.5749e-03, 1.0034e-03,
        2.6220e-03, 1.5502e-04, 5.2684e-02, 8.1633e-03, 8.9640e-01, 2.3149e-02,
        4.1019e-03, 1.4415e-03, 1.7408e-03]))
----------------------------------------
predict_3: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([7.0797e-03, 1.7802e-03, 2.1385e-03, 3.2882e-04, 1.5749e-03, 1.0034e-03,
        2.6220e-03, 1.5502e-04, 5.2684e-02, 8.1633e-03, 8.9640e-01, 2.3149e-02,
        4.1019e-03, 1.4415e-03, 1.7408e-03]))
----------------------------------------
predict_4: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([7.0797e-03, 1.7802e-03, 2.1385e-03, 3.2882e-04, 1.5749e-03, 1.0034e-03,
        2.6220e-03, 1.5502e-04, 5.2684e-02, 8.1633e-03, 8.9640e-01

## Test on Valid data

In [29]:
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)
random_test(data.valid_ds, learn)

image num: 8531
actual: No Finding
predict_2: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([0.0511, 0.0087, 0.0130, 0.0038, 0.0206, 0.0121, 0.0175, 0.0014, 0.1106,
        0.0468, 0.6689, 0.0729, 0.0227, 0.0078, 0.0139]))
----------------------------------------
predict_3: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([0.0511, 0.0087, 0.0130, 0.0038, 0.0206, 0.0121, 0.0175, 0.0014, 0.1106,
        0.0468, 0.6689, 0.0729, 0.0227, 0.0078, 0.0139]))
----------------------------------------
predict_4: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), tensor([0.0511, 0.0087, 0.0130, 0.0038, 0.0206, 0.0121, 0.0175, 0.0014, 0.1106,
        0.0468, 0.6689, 0.0729, 0.0227, 0.0078, 0.0139]))
----------------------------------------
predict_5: (MultiCategory No Finding, tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.

In [30]:
#img = data.train_ds[0][0]
#learn.predict(img, thresh = 0.2)

In [31]:
#from sklearn.metrics import roc_auc_score

In [32]:
#def auroc_score(input, target):
    #input, target = input.cpu().numpy(), target.cpu().numpy()
    #return roc_auc_score(target, input)

In [33]:
def get_labelled_preds(preds, p_thresh):
    labelled_preds = [';'.join([learn.data.classes[i] for i,p in enumerate(pred) if p > p_thresh]) for pred in preds]
    cleaned_labelled_preds = []
    for l in labelled_preds:
        lx = l
        if (l != "No Finding"):
            lx = l.replace("No Finding;", "")
        if (l != "No Finding"):
            lx = lx.replace("No Finding", "")
        cleaned_labelled_preds.append(lx)
    #actual = list(learn.data.valid_ds.y)
    #act = []
    #for a in actual:
    #     act.append(str(a).replace('Multicategory ', ''))
    return(cleaned_labelled_preds, labelled_preds)

In [34]:
# Cleaning of Labels happening here.
#labelled_preds = [';'.join([learn.data.classes[i] for i,p in enumerate(pred) if p > p_thresh]) for pred in preds]
#cleaned_labelled_preds = []
#for l in labelled_preds:
    #lx = l
    #if (l != "No Finding"):
     #   lx = l.replace("No Finding;", "")
    #if (l != "No Finding"):
      #  lx = lx.replace("No Finding", "")
    #cleaned_labelled_preds.append(lx)

#actual = list(learn.data.valid_ds.y)
#act = []
#for a in actual:
     #act.append(str(a).replace('Multicategory ', ''))

In [35]:
y_actuals = []
for i in data.valid_ds.y:
    y_actuals.append(str(i).replace('Multicategory ', ''))

In [36]:
df_y = pd.DataFrame({'actual': y_actuals})
#df_y

In [37]:
cleaned_02, predicted_02 = get_labelled_preds(preds, p_thresh = 0.2)
cleaned_03, predicted_03 = get_labelled_preds(preds, p_thresh = 0.3)
cleaned_04, predicted_04 = get_labelled_preds(preds, p_thresh = 0.4)
cleaned_05, predicted_05 = get_labelled_preds(preds, p_thresh = 0.5)

In [38]:
#df_1 = pd.DataFrame({'actual':act, 'predicted':labelled_preds,  'cleaned':cleaned_labelled_preds}, columns=['actual', 'predicted','cleaned'])


In [39]:
df_1 = pd.DataFrame({'cleaned_02':cleaned_02, 'cleaned_03':cleaned_03,  'predicted_02':predicted_02, 'predicted_03':predicted_03}, columns=['cleaned_02', 'cleaned_03','predicted_02', 'predicted_03'])


In [40]:
df_1a = pd.DataFrame({'cleaned_04':cleaned_04, 'cleaned_05':cleaned_05,  'predicted_04':predicted_04, 'predicted_05':predicted_05}, columns=['cleaned_04', 'cleaned_05','predicted_04', 'predicted_05'])


In [41]:
arr = preds.data.cpu().numpy()
#arr.shape
df_2 = pd.DataFrame(arr)
#Assigning the column names to classes.
df_2.columns = learn.data.classes
df_results = pd.concat([df_y,df_1,df_1a, df_2],axis = 1); df_results
df_results.to_csv('error_analysis-v2.csv', index=False)
df_results

,actual,cleaned_02,cleaned_03,predicted_02,predicted_03,cleaned_04,cleaned_05,predicted_04,predicted_05,Atelectasis,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,No Finding,,,,,,,,,0.069838,...,0.061618,0.061439,0.061145,0.073706,0.062626,0.104973,0.063411,0.061823,0.061864,0.061819
1,Infiltration,,,,,,,,,0.064924,...,0.061770,0.062501,0.061337,0.070319,0.063322,0.112226,0.064769,0.062744,0.061820,0.062609
2,No Finding,,,,,,,,,0.060712,...,0.060565,0.060619,0.060510,0.066792,0.060905,0.143744,0.061710,0.060730,0.060603,0.060733
3,Infiltration;Nodule;Pneumothorax,,,,,,,,,0.074970,...,0.062327,0.061238,0.059241,0.069827,0.065941,0.075654,0.063858,0.065733,0.059715,0.068496
4,No Finding,,,,,,,,,0.066651,...,0.061691,0.061742,0.061150,0.065391,0.063087,0.123630,0.063969,0.062045,0.061350,0.061689
5,No Finding,,,,,,,,,0.067772,...,0.060826,0.060589,0.060497,0.080899,0.062500,0.087607,0.061978,0.061168,0.061572,0.061021
6,Effusion;Infiltration,,,,,,,,,0.077498,...,0.061973,0.061455,0.061189,0.068348,0.062960,0.107268,0.063336,0.062009,0.061490,0.061914
7,Pneumothorax,,,,,,,,,0.065572,...,0.062471,0.062297,0.060605,0.076461,0.064429,0.089454,0.065925,0.063219,0.062067,0.066737
8,No Finding,,,,,,,,,0.062319,...,0.061296,0.061437,0.060979,0.069259,0.062392,0.128562,0.063616,0.061764,0.061215,0.061837
9,No Finding,,,,,,,,,0.060970,...,0.060599,0.060883,0.060495,0.063196,0.061428,0.145129,0.062683,0.060992,0.060574,0.060692


### EXPORTING MODEL AND OTHER INFORMATION

In [42]:
import os
import io
import tarfile
import PIL
import boto3
from fastai.vision import *

IMAGE_SIZE = 128 # Should match with the size specified while building the model.
learn.save("chestxray-model")
source_file = learn.model_dir + "chestxray-model" + ".pth"
dest_file = "chestxray-model" + ".pth"
shutil.copy2(source_file, dest_file)
save_texts('classes.txt', data.classes)
trace_input = torch.ones(1,3,IMAGE_SIZE,IMAGE_SIZE).cuda()
jit_model = torch.jit.trace(learn.model.float(), trace_input)
model_file='chestxray_resnet50_jit.pth'
output_path = str({model_file})
#torch.jit.save(jit_model, output_path)
torch.jit.save(jit_model, 'chestxray_resnet50_jit.pth')
tar_file='chestxray-v2.tar.gz'
classes_file='classes.txt'

print(os.listdir("."))
with tarfile.open(tar_file, 'w:gz') as f:
    f.add(model_file, arcname=model_file)
    f.add(classes_file, arcname=classes_file)

['error_analysis-v2.csv', 'classes.txt', 'chestxray-model.pth', '__output__.json', 'error_info_2.csv', 'chestxray_resnet50_jit.pth', '__notebook__.ipynb']
